In [4]:
import pandas as pd
import numpy as np
# m1 = pd.read_csv('modeling1.csv')
# m2 = pd.read_csv('modeling2.csv')
m3 = pd.read_csv('modeling3.csv')
m4 = pd.read_csv('modeling4.csv')
!pip install catboost

Defaulting to user installation because normal site-packages is not writeable


#머신러닝닝

In [5]:
m4

,충전실온도,실링온도,쿠킹온도,쿠킹스팀압력,실링압력,오류발생여부
0,71.125831,93.425831,127.878281,22.968,201.430,1
1,71.587470,103.987470,138.736120,23.166,204.974,1
2,70.540995,102.440995,137.135195,23.129,204.249,1
3,71.401189,111.901189,146.585739,23.123,200.068,1
4,71.428486,125.128486,159.591086,22.975,198.316,1
...,...,...,...,...,...,...
1069,69.827929,86.927929,121.388229,22.974,200.317,1
1070,71.671267,112.571266,147.339967,23.179,211.102,1
1071,71.764168,108.664168,142.862968,22.799,201.039,1
1072,72.585609,93.885609,128.549109,23.109,204.160,1


In [11]:
#input target split
y = m4['오류발생여부']
X = m4.drop(['오류발생여부'], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# #scale -> 넣고 안넣고 잘 선택 필요요
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# scaler = StandardScaler()
# #scaler = MinMaxScaler()
# scaler.fit(X_train)
# X_train_scaled = scaler.transform(X_train)
# X_test_scaled = scaler.transform(X_test)

#fit
# from sklearn.linear_model import LogisticRegression
#m1 = XGBClassifier()

# from sklearn.ensemble import RandomForestClassifier
# m1 = RandomForestClassifier(max_depth=10)

# from lightgbm import LGBMClassifier
# m1 = LGBMClassifier()

from catboost import CatBoostClassifier
m03 = CatBoostClassifier()

m03.fit(X_train, y_train)

CatBoostError: C:/Program Files (x86)/Go Agent/pipelines/BuildMaster/catboost.git/catboost/private/libs/target/target_converter.cpp:375: Target contains only one unique value

In [8]:
#m.score(X_train_scaled, y_train)

m03.score(X_train, y_train)

#m.score(X_test_scaled, y_test)
m03.score(X_test, y_test)

y_pred_m3 = m03.predict(X_test)

from sklearn.metrics import classification_report
report_m3 = classification_report(y_test, y_pred_m3)
importance_m3 = m03.feature_importances_
importance_m3

CatBoostError: There is no trained model to use score(). Use fit() to train model. Then use this method.

In [ ]:
print(m03.score(X_train, y_train))
print(m03.score(X_test, y_test))
print(report_m3)

0.999948003327787
0.9997920565606155
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4594
           1       1.00      1.00      1.00       215

    accuracy                           1.00      4809
   macro avg       1.00      1.00      1.00      4809
weighted avg       1.00      1.00      1.00      4809



In [7]:
import plotly.graph_objects as go
fig = go.Figure([go.Bar(x=X.columns, y=importance_m3)])
fig.update_layout(width=600)
fig.show()

NameError: name 'importance_m3' is not defined

#공정별 오류 분류

#딥러닝닝

In [25]:
#input target split
y = m3['Error_class']
X = m3.drop(['Error_class'], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

#스케일
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#tensorflow import
import tensorflow as tf
#from tensorflow import keras 잘 안씀
from tensorflow.keras import layers


#모델
model = tf.keras.Sequential([
layers.Dense(128, activation='relu', input_dim=5, name='Input_data-128-relu-dim5'), #64 = 퍼셉트론 64개 #input_shape = (6,0)
layers.Dense(64, activation='relu', name='D2-64-relu'), #64 = 퍼셉트론 64개, 레이어를 원하는대로 늘릴 수 있음 , 해보기 전 까지 알 수 없음 
#layers.Dropout(0.2), #다음 레이어로 넘어갈 때, 20%만큼의 데이터를 날림 --> 오버피팅 방지(모데륙제제)
layers.Dense(32, activation='relu', name='D3-32-relu'),
layers.Dense(4, activation = 'softmax', name='softmax-4')  #softmax(확률로 4개의 분류로 나타내 줌), 다중분류
])
model.summary()


#컴파일
optimizer = tf.keras.optimizers.RMSprop(0.001) #(0.001) = learning rate

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_data-128-relu-dim5 (D  (None, 128)              768       
 ense)                                                           
                                                                 
 D2-64-relu (Dense)          (None, 64)                8256      
                                                                 
 D3-32-relu (Dense)          (None, 32)                2080      
                                                                 
 softmax-4 (Dense)           (None, 4)                 132       
                                                                 
Total params: 11,236
Trainable params: 11,236
Non-trainable params: 0
_________________________________________________________________


In [26]:
#트레이닝
history = model.fit(X_train_scaled, y_train, epochs=200, validation_split = 0.2, batch_size = 24)

Epoch 1/200
29/29 [==============================] - 1s 10ms/step - loss: 1.0033 - accuracy: 0.6652 - val_loss: 0.8560 - val_accuracy: 0.7035
Epoch 2/200
29/29 [==============================] - 0s 4ms/step - loss: 0.8088 - accuracy: 0.7089 - val_loss: 0.8219 - val_accuracy: 0.7035
Epoch 3/200
29/29 [==============================] - 0s 3ms/step - loss: 0.7954 - accuracy: 0.7089 - val_loss: 0.8206 - val_accuracy: 0.7035
Epoch 4/200
29/29 [==============================] - 0s 3ms/step - loss: 0.7874 - accuracy: 0.7089 - val_loss: 0.8482 - val_accuracy: 0.7035
Epoch 5/200
29/29 [==============================] - 0s 8ms/step - loss: 0.7900 - accuracy: 0.7089 - val_loss: 0.8092 - val_accuracy: 0.7035
Epoch 6/200
29/29 [==============================] - 0s 5ms/step - loss: 0.7825 - accuracy: 0.7089 - val_loss: 0.8158 - val_accuracy: 0.7035
Epoch 7/200
29/29 [==============================] - 0s 4ms/step - loss: 0.7838 - accuracy: 0.7089 - val_loss: 0.8065 - val_accuracy: 0.7035
Epoch 8/200


In [27]:
history.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [28]:
import plotly.graph_objects as go

epochs=list(range(1, len(history.history['accuracy'])+1))
train_loss = history.history['accuracy']
val_loss =  history.history['val_accuracy']

fig = go.Figure(data=go.Scatter(x=epochs,
                                y=history.history['accuracy'],
                                name = 'Train Accuracy'))
fig.add_trace(go.Scatter(x=epochs, y=history.history['val_accuracy'],
                        name = 'Validation Accuracy'))

fig.update_xaxes(title_text="epoch")
fig.update_yaxes(title_text="accuracy")
fig.update_layout(legend=dict(yanchor="top", y=1.15, xanchor="left", x=0.75), 
                  title_text="Model Evaluation", title_x=0.5, width=800)
fig.show()

In [29]:
model.evaluate(X_test_scaled, y_test)

7/7 [==============================] - 0s 2ms/step - loss: 0.9178 - accuracy: 0.6837


[0.9177770614624023, 0.6837209463119507]

In [30]:
epochs = list(range(1, len(history.history['accuracy']) + 1))
train_accuracy = history.history['accuracy']
import plotly.graph_objects as go

fig = go.Figure(data=go.Scatter(x=epochs,
                                y=train_accuracy,
                                name = 'train_accuracy'))
fig.show()

In [31]:
from tensorflow.keras.utils import plot_model
plot_model(model, 'softmax.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


#2진

In [32]:
#input target split
y = m4['오류발생여부']
X = m4.drop(['오류발생여부'], axis = 1)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

#스케일
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

#tensorflow import
import tensorflow as tf
#from tensorflow import keras 잘 안씀
from tensorflow.keras import layers


#모델
model1 = tf.keras.Sequential([
layers.Dense(128, activation='relu', input_dim=5, name='128-relu-dim5'), #64 = 퍼셉트론 64개 #input_shape = (6,0)
layers.Dense(64, activation='relu', name='D2-64-relu'), #64 = 퍼셉트론 64개, 레이어를 원하는대로 늘릴 수 있음 , 해보기 전 까지 알 수 없음 
#layers.Dropout(0.2), #다음 레이어로 넘어갈 때, 20%만큼의 데이터를 날림 --> 오버피팅 방지(모데륙제제)
layers.Dense(32, activation='relu', name='D3-32-relu'),
layers.Dense(2, activation = 'softmax', name='softmax-2or4')  #softmax(확률로 4개의 분류로 나타내 줌), 다중분류
])
model1.summary()


#컴파일
optimizer = tf.keras.optimizers.RMSprop(0.001) #(0.001) = learning rate

model1.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 128-relu-dim5 (Dense)       (None, 128)               768       
                                                                 
 D2-64-relu (Dense)          (None, 64)                8256      
                                                                 
 D3-32-relu (Dense)          (None, 32)                2080      
                                                                 
 softmax-2or4 (Dense)        (None, 2)                 66        
                                                                 
Total params: 11,170
Trainable params: 11,170
Non-trainable params: 0
_________________________________________________________________


In [33]:
#트레이닝
history = model1.fit(X_train_scaled, y_train, epochs=100, validation_split = 0.2, batch_size = 24)

Epoch 1/100
29/29 [==============================] - 1s 11ms/step - loss: 0.1672 - accuracy: 0.9956 - val_loss: 0.0146 - val_accuracy: 1.0000
Epoch 2/100
29/29 [==============================] - 0s 8ms/step - loss: 0.0052 - accuracy: 1.0000 - val_loss: 0.0016 - val_accuracy: 1.0000
Epoch 3/100
29/29 [==============================] - 0s 6ms/step - loss: 7.0485e-04 - accuracy: 1.0000 - val_loss: 3.5038e-04 - val_accuracy: 1.0000
Epoch 4/100
29/29 [==============================] - 0s 4ms/step - loss: 1.6846e-04 - accuracy: 1.0000 - val_loss: 1.2217e-04 - val_accuracy: 1.0000
Epoch 5/100
29/29 [==============================] - 0s 4ms/step - loss: 6.3788e-05 - accuracy: 1.0000 - val_loss: 5.7817e-05 - val_accuracy: 1.0000
Epoch 6/100
29/29 [==============================] - 0s 3ms/step - loss: 3.3529e-05 - accuracy: 1.0000 - val_loss: 3.6483e-05 - val_accuracy: 1.0000
Epoch 7/100
29/29 [==============================] - 0s 4ms/step - loss: 2.2240e-05 - accuracy: 1.0000 - val_loss: 2.6336

In [34]:
import plotly.graph_objects as go

epochs=list(range(1, len(history.history['accuracy'])+1))
train_loss = history.history['accuracy']
val_loss =  history.history['val_accuracy']

fig = go.Figure(data=go.Scatter(x=epochs,
                                y=history.history['accuracy'],
                                name = 'Train Accuracy'))
fig.add_trace(go.Scatter(x=epochs, y=history.history['val_accuracy'],
                        name = 'Validation Accuracy'))

fig.update_xaxes(title_text="epoch")
fig.update_yaxes(title_text="accuracy")
fig.update_layout(legend=dict(yanchor="top", y=1.15, xanchor="left", x=0.75), 
                  title_text="Model Evaluation", title_x=0.5, width=800)
fig.show()

In [21]:
model1.evaluate(X_test_scaled, y_test)

7/7 [==============================] - 0s 2ms/step - loss: 5.8939e-07 - accuracy: 1.0000


[5.893885486329964e-07, 1.0]

151/151 [==============================] - 1s 4ms/step - loss: 0.1816 - accuracy: 0.9551
0.18157784640789032
0.9550842046737671


In [22]:
from tensorflow.keras.utils import plot_model
plot_model(model1, 'softmax.png')

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


151/151 [==============================] - 1s 3ms/step


In [24]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# 성능 지표 계산
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# 데이터프레임 생성
df = pd.DataFrame({
    'Metric': ['Accuracy', 'Precision', 'Recall', 'F1 Score'],
    'Score': [accuracy, precision, recall, f1]
})

NameError: name 'y_true' is not defined